In [1]:
import random
import itertools
from IPython.display import clear_output
import time
import pandas as pd

# General

### Card Deck Representation

In [2]:
ranks = ['Two', 'Three', 'Four', 'Five', 'Six', 'Seven', 'Eight', 'Nine', 'Ten', 'Jack', 'Queen', 'King', 'Ace']
suits = ['Spades', 'Clubs', 'Hearts', 'Diamonds']
values = {'Two':2, 'Three':3, 'Four':4, 'Five':5, 'Six':6, 'Seven':7, 'Eight':8, 
          'Nine':9, 'Ten':10, 'Jack':11, 'Queen':12, 'King':13, 'Ace': 14}
value_to_rank = {'2': 'Two', '3': 'Three', '4': 'Four', '5': 'Five', '6': 'Six', '7': 'Seven', '8': 'Eight', 
                 '9': 'Nine', 'T': 'Ten', 'J': 'Jack', 'Q': 'Queen', 'K': 'King', 'A': 'Ace'} # for range input
possible_hands = ['royal flush', 'straight flush', 'four of a kind', 'full house', 'flush', 'straight', 'three of a kind', 'two pair', 'one pair', 'high card']

In [3]:
class Card:
    
    def __init__(self, rank, suit):
        self.rank = rank
        self.suit = suit
        self.value = values[self.rank]
    
    def __repr__(self):
        return self.rank + ' ' + self.suit
    
    def __str__(self):
        return self.rank + ' ' + self.suit

In [4]:
class Deck:
    
    def __init__(self, variant = 'holdem'):
        self.deck = []
        if variant == "Short Deck":
            for suit in suits:
                for rank in ranks[4:]:
                    self.deck.append(Card(rank, suit))
        else:
            for suit in suits:
                for rank in ranks:
                    self.deck.append(Card(rank, suit))
                
    def shuffle(self):
        random.shuffle(self.deck)
    
    def burn(self):
        return self.deck.pop(0)
    
    def deal(self):
        return self.deck.pop(0)
    
    def remove(self, remove_card): # remove specific card
        for card in self.deck:
            if str(card) == str(remove_card):
                self.deck.remove(card)
    
    def __str__(self):
        deck = ''
        for card in self.deck:
            deck += card.__str__() + '\n'
        return deck[:-1]

### Player Representation

In [5]:
class Player:
    
    def __init__(self, name, chips, hole_cards, in_hand, not_acted, curr_bet, all_in):
        self.name = name
        self.chips = chips
        self.hole_cards = hole_cards
        self.in_hand = in_hand
        self.not_acted = not_acted
        self.curr_bet = curr_bet
        self.all_in = all_in
        
    def fold(self):
        self.in_hand = False
        self.curr_bet = 0
    
    def bet(self, amnt):
        self.chips -= amnt
        self.curr_bet += amnt
        
    def chip_transaction(self, amnt):
        self.chips += amnt
    
    def __repr__(self):
        return self.name
    
    def __str__(self):
        return self.name

In [6]:
def count_players_in_hand(players):
    count = 0
    for player in players:
        if player.in_hand:
            count += 1
    return count

### Hand Strength Detection (at showdown)

In [7]:
def best_hand(cards, variant = 'holdem'):
    possible_hands = []
    if variant == 'Omaha':
        hole_cards = cards[:4]
        community_cards = cards[4:]
        for hand in list(itertools.combinations(cards, 5)):
            hole_hand = [card for card in hand if card in hole_cards]
            community_hand = [card for card in hand if card in community_cards]
            if len(hole_hand) == 2 and len(community_hand) == 3:
                possible_hands.append(list(hand))
    else:
        for hand in list(itertools.combinations(cards, 5)):
            possible_hands.append(list(hand))
    possible_hands.sort(key = (lambda x: x[0].value + x[1].value + x[2].value + x[3].value + x[4].value), reverse = True)
    for hand in possible_hands:
        hand.sort(key = (lambda x: x.value), reverse = True) 
    
    # royal flush and straight flush
    for hand in possible_hands:
        if hand[0].suit == hand[1].suit == hand[2].suit == hand[3].suit == hand[4].suit:
            if hand[0].rank == 'Ace' and hand[1].rank == 'King' and hand[2].rank == 'Queen' and hand[3].rank == 'Jack' and hand[4].rank == 'Ten':
                return "royal flush", [hand[0], hand[1], hand[2], hand[3], hand[4]]
            elif hand[0].value == (hand[1].value + 1) == (hand[2].value + 2) == (hand[3].value + 3) == (hand[4].value + 4):
                return "straight flush", [hand[0], hand[1], hand[2], hand[3], hand[4]]
            
    # quads
    for hand in possible_hands:
        if hand[0].rank == hand[1].rank == hand[2].rank == hand[3].rank:
            return "four of a kind", [hand[0], hand[1], hand[2], hand[3], hand[4]]
        if hand[1].rank == hand[2].rank == hand[3].rank == hand[4].rank:
            return "four of a kind", [hand[1], hand[2], hand[3], hand[4], hand[0]]
    
    if variant == 'Short Deck': # flush beats full house in short deck
        # flush
        for hand in possible_hands:
            if hand[0].suit == hand[1].suit == hand[2].suit == hand[3].suit == hand[4].suit:
                return "flush", [hand[0], hand[1], hand[2], hand[3], hand[4]]
    
        # full house
        for hand in possible_hands:
            if (hand[0].rank == hand[1].rank) and (hand[2].rank == hand[3].rank == hand[4].rank):
                return "full house", [hand[2], hand[3], hand[3], hand[0], hand[1]]
            if (hand[0].rank == hand[1].rank == hand[2].rank) and (hand[3].rank == hand[4].rank):
                return "full house", [hand[0], hand[1], hand[2], hand[3], hand[4]]
    
    else: # full house beats flush in other variants
        # full house
        for hand in possible_hands:
            if (hand[0].rank == hand[1].rank) and (hand[2].rank == hand[3].rank == hand[4].rank):
                return "full house", [hand[2], hand[3], hand[3], hand[0], hand[1]]
            if (hand[0].rank == hand[1].rank == hand[2].rank) and (hand[3].rank == hand[4].rank):
                return "full house", [hand[0], hand[1], hand[2], hand[3], hand[4]]

        # flush
        for hand in possible_hands:
            if hand[0].suit == hand[1].suit == hand[2].suit == hand[3].suit == hand[4].suit:
                return "flush", [hand[0], hand[1], hand[2], hand[3], hand[4]]
    
    # straight
    for hand in possible_hands:
        if hand[0].value == (hand[1].value + 1) == (hand[2].value + 2) == (hand[3].value + 3) == (hand[4].value + 4):
            return "straight", [hand[0], hand[1], hand[2], hand[3], hand[4]]
        elif hand[0].rank == 'Ace' and hand[1].rank == 'Six' and hand[2].rank == 'Seven' and hand[3].rank == 'Eight' and hand[4].rank == 'Nine':
            if variant == 'Short Deck':
                return "straight", [hand[0], hand[1], hand[2], hand[3], hand[4]]
        elif hand[0].rank == 'Ace' and hand[1].rank == 'Five' and hand[2].rank == 'Four' and hand[3].rank == 'Three' and hand[4].rank == 'Two':
            return "straight", [hand[0], hand[1], hand[2], hand[3], hand[4]]
        
    # trips
    for hand in possible_hands:
        if hand[0].rank == hand[1].rank == hand[2].rank:
            return "three of a kind", [hand[0], hand[1], hand[2], hand[3], hand[4]]
        if hand[1].rank == hand[2].rank == hand[3].rank:
            return "three of a kind", [hand[1], hand[2], hand[3], hand[0], hand[4]]
        if hand[2].rank == hand[3].rank == hand[4].rank:
            return "three of a kind", [hand[2], hand[3], hand[4], hand[0], hand[1]]
        
    # two pair
    for hand in possible_hands:
        if (hand[0].rank == hand[1].rank) and (hand[2].rank == hand[3].rank):
            return "two pair", [hand[0], hand[1], hand[2], hand[3], hand[4]]
        if (hand[0].rank == hand[1].rank) and (hand[3].rank == hand[4].rank):
            return "two pair", [hand[0], hand[1], hand[3], hand[4], hand[2]]
        if (hand[1].rank == hand[2].rank) and (hand[3].rank == hand[4].rank):
            return "two pair", [hand[1], hand[2], hand[3], hand[4], hand[0]]
        
    # one pair
    for hand in possible_hands:
        if hand[0].rank == hand[1].rank:
            return "one pair", [hand[0], hand[1], hand[2], hand[3], hand[4]]
        if hand[1].rank == hand[2].rank:
            return "one pair", [hand[1], hand[2], hand[0], hand[3], hand[4]]
        if hand[2].rank == hand[3].rank:
            return "one pair", [hand[2], hand[3], hand[0], hand[1], hand[4]]
        if hand[3].rank == hand[4].rank:
            return "one pair", [hand[3], hand[4], hand[0], hand[1], hand[2]]
        
    hand = possible_hands[0]
    return 'high card', [hand[0], hand[1], hand[2], hand[3], hand[4]]

### Gameplay Functionality

In [8]:
def hole_cards(players, deck, variant = 'holdem'):
    num_hole_cards = 2
    if variant == 'Omaha':
        num_hole_cards = 4
    for i in range(num_hole_cards): # deal hole cards to each player
        for player in players:
            player.hole_cards.append(deck.deal())
    for player in players: # view cards
        view = input(f"Press enter to view {player.name}'s cards.")
        print(player.hole_cards)
        exit = input("Press enter to stop viewing.")
        clear_output()
        time.sleep(0.05)
    return players, deck

In [9]:
def betting_round(players, starting_position, curr_bet, pot, bb):
    
    i = starting_position # index of first player to act
    
    betting = True
    while betting:
        
        player = players[i]
        
        # player action
        if player.in_hand and player.not_acted and not player.all_in:
            invalid_action = True
            while invalid_action:
                
                print(f"{player.name} Action: ")
                move = input("Enter one of 'fold' / 'check' / 'call' / 'raise' / 'all-in': ")
                
                # fold
                if move[0] == 'f':
                    invalid_action = False
                    player.fold()
                    player.not_acted = False
                
                # check and call
                elif move[0] == 'c':    
                    if move[1] == 'h': # check
                        if player.curr_bet == curr_bet:
                            invalid_action = False
                            player.not_acted = False
                        else:
                            print("Invalid Action; You must match current bet or fold.") 
                    else: # call
                        if player.curr_bet < curr_bet and player.chips >= (curr_bet - player.curr_bet):
                            invalid_action = False
                            pot += (curr_bet - player.curr_bet)
                            player.bet(curr_bet - player.curr_bet)
                            player.not_acted = False
                        elif player.curr_bet < curr_bet and player.chips < (curr_bet - player.curr_bet):
                            move = 'all in' # if insufficient chips to call, player shoves
                        else: # if player calls when no one has bet, player checks
                            invalid_action = False
                            player.not_acted = False
                            
                # raise
                elif move[0] == 'r':
                    if curr_bet > 0 and player.chips <= (2*curr_bet):
                        move = 'a'
                    elif curr_bet == 0 and player.chips <= bb:
                        move = 'a'
                    else:
                        invalid_action = False
                        invalid_raise = True
                        while invalid_raise:
                            amnt = int(input('Enter total number of chips to bet(includes current bet): '))
                            if curr_bet > 0:
                                if amnt >= (2*curr_bet):
                                    invalid_raise = False
                                    pot += (amnt - player.curr_bet)
                                    player.bet(amnt - player.curr_bet)
                                    curr_bet = amnt
                                    player.not_acted = False
                                    for player in players:
                                        if player.in_hand and player != players[i] and not(player.all_in):
                                            player.not_acted = True 
                                else:
                                    print('Invalid Raise Amount; You must at least double current bet.')
                            else:
                                if amnt >= bb:
                                    invalid_raise = False
                                    pot += (amnt - player.curr_bet)
                                    player.bet(amnt)
                                    curr_bet = amnt
                                    player.not_acted = False
                                    for player in players:
                                        if player.in_hand and player != players[i] and not(player.all_in):
                                            player.not_acted = True        
                                else:
                                    print('Invalid Raise Amount; Your bet must be greater than or equal to big blind.')
                
                elif move[0] != 'a':
                    print('Invalid Action')
                
                if move[0] == 'a': # all in
                    invalid_action = False
                    curr_bet_updated = False
                    if (player.chips + player.curr_bet) > curr_bet:
                        curr_bet = player.chips + player.curr_bet
                        curr_bet_updated = True
                    pot += player.chips
                    player.bet(player.chips)
                    player.all_in = True
                    player.not_acted = False
                    if curr_bet_updated:
                        for player in players:
                            if player.in_hand and player != players[i] and not(player.all_in):
                                player.not_acted = True
        
        # after each action, check if betting is finished
        betting = False
        if count_players_in_hand(players) > 1:
            for player in players:
                if player.not_acted: # CONTINUE FROM HERE
                    betting = True
                    break
        
        # move to next player
        if players[i] == players[-1]:
            i = 0
        else:
            i += 1
        
    clear_output()
    time.sleep(0.05)
        
    return players, pot

In [10]:
def community_cards(cards, num, deck, pot):
    deck.burn()
    for i in range(num):
        cards.append(deck.deal())
    print(cards)
    print(f"Pot: {pot}\n")
    return cards, deck

In [11]:
def reset_curr_bets(players):
    count = 0
    for player in players:
        if player.in_hand and not player.all_in:
            count += 1
    if count == 1:
        return players
    for player in players:
        if player.in_hand:
            player.not_acted = True
            player.curr_bet = 0
    return players

In [12]:
def detect_winner(round_, players, pot): # if everyone has folded, declare last man standing as winner
    if count_players_in_hand(players) == 1:
        round_ = False
        for player in players:
            if player.in_hand:
                print(player.name + " wins " + str(pot) + " chips.")
                player.chip_transaction(pot)
    return round_, players

In [13]:
def showdown_winner(players, river, pot, variant):
    if count_players_in_hand(players) != 1:
        hands = {}
        for player in players:
            if player.in_hand:
                cards = player.hole_cards + river
                value_lst = [] # convert list of card to list of values of cards
                for card in best_hand(cards, variant)[1]:
                    value_lst.append(card.value)
                hands[player] = ((best_hand(cards, variant)[0], value_lst, best_hand(cards, variant)[1]), possible_hands.index(best_hand(cards, variant)[0]))
        best = min(hands.items(), key = (lambda x: x[1][1]))[1][1]
        for player in hands.keys():
            if hands[player][1] != best:
                hands[player] = (('0', [0, 0, 0, 0, 0]), 10) # effectively deletes this player
        
        # finding best kicker (sorted by value_lst)
        winner = sorted(hands.items(), key = (lambda x: x[1][0][1]), reverse = True)[0][0]

        # reporting winner
        print(f"{winner.name} hole cards are {winner.hole_cards}.")
        print(f"Community cards are {river}.\n")
        print(f"{winner.name}'s hand: {hands[winner][0][2]}")
        print(winner.name + " wins " + str(pot) + " chips with " + hands[winner][0][0])
        winner.chip_transaction(pot)
        
    return players

In [14]:
def play_again(players, game):
    for player in players:
        if player.chips == 0:
            players.remove(player)
    if len(players) == 1:
        print(f"\n{players[0].name} Wins With {players[0].chips}!")
        print("Thanks for playing!")
        return players, False
    again = input("\nDo you want to play again?")
    if again.lower()[0] == 'y':
        game = True
        players = players[1:] + [players[0]] # position progression
        for player in players: # reset
            player.hole_cards = []
            player.in_hand = True
            player.not_acted = True
            player.curr_bet = 0
            player.all_in = False
        clear_output()
        time.sleep(0.05)
    else:
        game = False
        clear_output()
        time.sleep(0.1)
        for player in players:
            print(f"{player.name}: {player.chips}")
        print("Thanks for playing!")
    return players, game

### Textual Interface for Game

In [15]:
variant = ''
while variant not in ['holdem', 'Omaha', 'Short Deck']:
    print('You can choose to play one of the following three poker variants: holdem, Omaha, Short Deck')
    variant = input("Which poker variant would you like to play?: ")
sb = int(input("Choose small blind amount (enter a number): "))
bb = sb*2
stack = int(input("Choose starting stack amount (enter a number): "))
num_players = int(input("How many players? (enter a number): "))
heads_up = False
if num_players == 2:
    heads_up = True

players = []
for player in range(num_players):
    name = input(f"Enter player {player}'s name: ")
    players.append(Player(name, stack, [], True, True, 0, False))
random.shuffle(players)

game = True
while game:
    
    round_ = True
    
    # positions
    print("Order of Action: ")
    for player in players:
        print(f"{player.name}: {player.chips}")
    continue_prompt = input("Press enter to continue: ")
    clear_output()
    time.sleep(0.05)
    
    # pre-flop
    if round_:
        deck = Deck(variant) # new deck
        deck.shuffle() # shuffle deck
        players, deck = hole_cards(players, deck, variant) # deal hole cards
        players[0].bet(sb) # small blind posted
        players[1].bet(bb) # big blind posted
        if heads_up:
            players, pot = betting_round(players, 0, bb, (sb + bb), bb) # round of betting
        else:
            players, pot = betting_round(players, 2, bb, (sb + bb), bb) # round of betting
        round_, players = detect_winner(round_, players, pot) # detects winner before showdown

    # flop
    if round_:
        flop, deck = community_cards([], 3, deck, pot) # deal flop
        players = reset_curr_bets(players) # reset current bets
        if heads_up:
            players, pot = betting_round(players, 1, 0, pot, bb) # round of betting
            print(players)
        else:
            players, pot = betting_round(players, 0, 0, pot, bb) # round of betting
        round_, players = detect_winner(round_, players, pot) # detects winner before showdown

    # turn
    if round_:
        turn, deck = community_cards(flop, 1, deck, pot) # deal turn
        players = reset_curr_bets(players) # reset current bets
        if heads_up:
            players, pot = betting_round(players, 1, 0, pot, bb) # round of betting
            print(players)
        else:
            players, pot = betting_round(players, 0, 0, pot, bb) # round of betting
        round_, players = detect_winner(round_, players, pot) # detects winner before showdown

    # river
    if round_:
        river, deck = community_cards(turn, 1, deck, pot) # deal river
        players = reset_curr_bets(players) # reset current bets
        if heads_up:
            players, pot = betting_round(players, 1, 0, pot, bb) # round of betting
        else:
            players, pot = betting_round(players, 0, 0, pot, bb) # round of betting
        round_, players = detect_winner(round_, players, pot) # detects winner before showdown
        players = showdown_winner(players, river, pot, variant) # determine best hand at showdown
    
    players, game = play_again(players, game) # another round

a hole cards are [Ten Spades, Five Spades].
Community cards are [Jack Hearts, Three Diamonds, Three Spades, Seven Diamonds, Eight Diamonds].

a's hand: [Three Diamonds, Three Spades, Jack Hearts, Ten Spades, Eight Diamonds]
a wins 400 chips with one pair

a Wins With 400!
Thanks for playing!


# Simulations

### Showdown Hand Probabilities

In [16]:
def showdown_hand_probability(num_iters, var):
    num_cards_in_hand = 7
    if var == 'Omaha':
        num_cards_in_hand = 9
    hand_history = []
    for x in range(num_iters):
        deck = Deck(variant = var)
        deck.shuffle()
        hand = []
        for x in range(num_cards_in_hand):
            hand.append(deck.deal())
        hand.sort(key = (lambda x: values[x.rank]), reverse = True)
        hand_history.append(best_hand(hand, variant = var)[0])
        
    probabilities = {'high card': hand_history.count('high card')/num_iters,
                     'one pair': hand_history.count('one pair')/num_iters, 
                     'two pair': hand_history.count('two pair')/num_iters, 
                     'three of a kind': hand_history.count('three of a kind')/num_iters, 
                     'straight': hand_history.count('straight')/num_iters,
                     'flush': hand_history.count('flush')/num_iters,
                     'full house': hand_history.count('full house')/num_iters,
                     'four of a kind': hand_history.count('four of a kind')/num_iters,
                     'straight flush': hand_history.count('straight flush')/num_iters,
                     'royal flush': hand_history.count('royal flush')/num_iters}
    return probabilities

In [17]:
num_iters = 1000000
holdem_probs = showdown_hand_probability(num_iters, 'holdem')
omaha_probs = showdown_hand_probability(num_iters, 'Omaha')
short_deck_probs = showdown_hand_probability(num_iters, 'Short Deck')

In [18]:
df = pd.DataFrame(index = possible_hands[::-1])
df['holdem'] = holdem_probs.values()
df['Omaha'] = omaha_probs.values()
df['Short Deck'] = short_deck_probs.values()
df.to_csv('poker_showdown_hand_probabilities.csv')

# Equity Calculator

Supports Hand vs Hand, Hand vs Range, and Range vs Range equity calculations, both pre-flop and post-flop

In [19]:
def hand_comparison(players, community_cards, var = 'holdem'):
    hands = {}
    for player in players:
        cards = player.hole_cards + community_cards
        value_lst = [] # convert list of card to list of values of cards
        for card in best_hand(cards, var)[1]:
            value_lst.append(card.value)
        hands[player] = ((best_hand(cards, var)[0], value_lst, best_hand(cards, var)[1]), possible_hands.index(best_hand(cards, var)[0]))
    best = min(hands.items(), key = (lambda x: x[1][1]))[1][1]
    for player in hands.keys():
        if hands[player][1] != best:
            hands[player] = (('0', [0, 0, 0, 0, 0]), 10) # effectively deletes this player
    winner = sorted(hands.items(), key = (lambda x: x[1][0][1]), reverse = True)[0][0] # finding best kicker
    return winner.name

In [20]:
def equity_calculator(r1, r2, cc = [], var = 'holdem', num_iters = 10000): # range vs range equity
    p1 = Player('p1', 0, [], True, True, 0, False)
    p2 = Player('p2', 0, [], True, True, 0, False)
    players = [p1, p2]
    p1_wins = 0
    for i in range(num_iters):
        p1.hole_cards = random.choice(r1) # r1 is list of lists (each element list is a hand)
        p2.hole_cards = random.choice(r2)
        deck = Deck(variant = var)
        community_cards = []
        for card in cc:
            community_cards.append(card)
        for card in p1.hole_cards:
            deck.remove(card)
        for card in p2.hole_cards:
            deck.remove(card)
        for card in community_cards:
            deck.remove(card)
        deck.shuffle()
        while(len(community_cards) != 5):
            community_cards.append(deck.deal())
        if hand_comparison(players, community_cards, var) == 'p1':
            p1_wins += 1
    print(f"Range 1: {round(p1_wins/num_iters * 100, 3)}%")
    print(f"Range 2: {round((1 - p1_wins/num_iters) * 100, 3)}%")
    return [(r1, round(p1_wins/num_iters * 100, 3)), (r2, round((1 - p1_wins/num_iters) * 100, 3))]

### Choosing Custom Hand Ranges

In [21]:
def pick_range(t = ''): # only for holdem
    range_ = []
    if t == '':
        r = input("Enter your range in standard format: ")
    else:
        r = t
    hands = r.split(',')
    count = 0
    for h in hands:
        if len(h) == 2: # pocket pair
            suit_combinations = list(itertools.combinations(suits, 2))
            for suit0, suit1 in suit_combinations:
                range_.append([Card(value_to_rank[h[0]], suit0), Card(value_to_rank[h[1]], suit1)])
                count += 1
        elif h[2] == 's': # suited
            for suit in suits:
                range_.append([Card(value_to_rank[h[0]], suit), Card(value_to_rank[h[1]], suit)])
                count += 1
        else: # off-suit
            suit_permutations = list(itertools.permutations(suits, 2))
            for suit0, suit1 in suit_permutations:
                range_.append([Card(value_to_rank[h[0]], suit0), Card(value_to_rank[h[1]], suit1)])
                count += 1
    # print(count, '\n')
    return range_

### Game Theory Optimal Button RFI Range for Heads Up NLH

In [22]:
def button_rfi():
    t = ''
    input_list = list(value_to_rank.keys())[::-1]
    for i in input_list:
        for j in input_list:
            if input_list.index(j) < input_list.index(i):
                continue
            t += i
            t += j
            if i == j:
                t += ','
            else:
                t += 's,'
    for i in ['A', 'K']:
        for j in input_list:
            if input_list.index(j) <= input_list.index(i):
                continue
            t += i
            t += j
            t += 'o,'
    for i in ['5', '4']:
        for j in input_list[:-1]:
            if input_list.index(j) <= input_list.index(i):
                continue
            t += i
            t += j
            t += 'o,'
    for i in ['Q', '7', '6']:
        for j in input_list[:-2]:
            if input_list.index(j) <= input_list.index(i):
                continue
            t += i
            t += j
            t += 'o,'
    for i in ['J', 'T', '9', '8']:
        for j in input_list[:-3]:
            if input_list.index(j) <= input_list.index(i):
                continue
            t += i
            t += j
            t += 'o,'
    t = t[:-1]
    return t

### Demonstrated Usage of Equity Calculator

In [23]:
# Using a custom range
# Instructions:
    # Input a number or letter from 2 to 9 or T (ten) to A (ace) for the first card in your hand. 
    # Input another number or letter in a similar format for the second card in your hand.
    # If your hand is a pocket pair, you are done. If not input o or s for off-suit or suited.
    # To input another card in your range, input a comma and then repeat. NO SPACES.
    
r1 = pick_range()
r2 = pick_range()

Enter your range in standard format: 65s
Enter your range in standard format: AA


In [24]:
# Using a default range
r1 = pick_range(button_rfi())
r2 = [[Card('Ace', 'Spades'), Card('Ace', 'Clubs')]]

In [25]:
lst = equity_calculator(r1, r2, [], 'holdem', 100000)

Range 1: 15.395%
Range 2: 84.605%


In [26]:
lst

[([[Ace Spades, Ace Clubs],
   [Ace Spades, Ace Hearts],
   [Ace Spades, Ace Diamonds],
   [Ace Clubs, Ace Hearts],
   [Ace Clubs, Ace Diamonds],
   [Ace Hearts, Ace Diamonds],
   [Ace Spades, King Spades],
   [Ace Clubs, King Clubs],
   [Ace Hearts, King Hearts],
   [Ace Diamonds, King Diamonds],
   [Ace Spades, Queen Spades],
   [Ace Clubs, Queen Clubs],
   [Ace Hearts, Queen Hearts],
   [Ace Diamonds, Queen Diamonds],
   [Ace Spades, Jack Spades],
   [Ace Clubs, Jack Clubs],
   [Ace Hearts, Jack Hearts],
   [Ace Diamonds, Jack Diamonds],
   [Ace Spades, Ten Spades],
   [Ace Clubs, Ten Clubs],
   [Ace Hearts, Ten Hearts],
   [Ace Diamonds, Ten Diamonds],
   [Ace Spades, Nine Spades],
   [Ace Clubs, Nine Clubs],
   [Ace Hearts, Nine Hearts],
   [Ace Diamonds, Nine Diamonds],
   [Ace Spades, Eight Spades],
   [Ace Clubs, Eight Clubs],
   [Ace Hearts, Eight Hearts],
   [Ace Diamonds, Eight Diamonds],
   [Ace Spades, Seven Spades],
   [Ace Clubs, Seven Clubs],
   [Ace Hearts, Seven Heart